In [ ]:
import pandas as pd
import numpy as np
import akshare as ak

#1 获取reits代码
def get_reits_code():
    df = ak.fund_name_em()
    df = df[df['基金类型'] == 'Reits']
    return df['基金代码'].tolist()

#2 获取单个REITS的分红送股
def get_fenhong(code):
    url = f'https://fundf10.eastmoney.com/fhsp_{code}.html'
    resp = pd.read_html(url)
    df1 = resp[1]
    if not df1.empty:
        df1['代码'] = code        
        df1['每份分红'] = df1['每份分红'].str.extract(r'([-+]?\d*\.?\d+)').astype(float)
        df1 = df1[['代码', '年份', '权益登记日', '除息日','分红发放日', '每份分红']] 
        df1['权益登记日'] = pd.to_datetime(df1['权益登记日'],errors='coerce',format='%Y-%m-%d')
        df1['除息日'] = pd.to_datetime(df1['除息日'],errors='coerce',format='%Y-%m-%d')
        df1['分红发放日'] = pd.to_datetime(df1['分红发放日'],errors='coerce',format='%Y-%m-%d')
    return df1

#3 获取所有REITS的分红送股
def get_all_fenhong():
    codes = get_reits_code()
    df = pd.DataFrame()
    for code in codes:
        df1 = get_fenhong(code)
        if not df1.empty:      
            df = pd.concat([df, df1])        
    return df

# 获取所有REITS的实时行情
def get_all_reits_price():
    df = ak.reits_realtime_em()
    return df

if __name__ == '__main__':
    df1 = get_all_reits_price()
    df2 = get_all_fenhong()
    with pd.ExcelWriter('output.xlsx',engine='openpyxl') as writer:
        df1.to_excel(writer, sheet_name='最新行情',index=False)
        df2.to_excel(writer, sheet_name='分红情况',index=False)
    print('Done')


Done


: 